# Snowflake Intelligence Hands-On Lab

In this session, you will learn how to:
- Perform semantic modeling to enable text-to-sql conversion via Cortex Analyst
- Build custom tools like web search
- Create Cortex Agents that can use Cortex Analyst and custom tools
- Use Snowflake Intelligence as a UI layer for Cortex Agents
- Investigate Snowflake's built in observability for the agent

## Section 1: Data prep

In this section, we'll configure the necessary Snowflake objects and data to perform the rest of the hands on lab.

For our structured data, we'll use a simple star schema, mocking insurance data. Our model will be composed of the following tables:
- fact_claims: our event table tracking claims that have taken place
- dim_policies: contains our information about the various policies issued
- dim_businesses: contains information about the various businesses that are insured

### 1.1 - Create tables

In [ ]:
CREATE OR REPLACE TABLE DIM_BUSINESS (
    BUSINESS_KEY INT PRIMARY KEY,
    BUSINESS_NAME VARCHAR(255),
    BUSINESS_CITY VARCHAR(100),
    BUSINESS_STATE VARCHAR(2),
    NAICS_CODE VARCHAR(10),
    INDUSTRY_SEGMENT VARCHAR(100),
    ASSET_VALUE_MILLIONS NUMERIC(10, 2)
);

CREATE OR REPLACE TABLE DIM_POLICY (
    POLICY_KEY INT PRIMARY KEY,
    BUSINESS_KEY INT REFERENCES DIM_BUSINESS (BUSINESS_KEY),
    POLICY_ID VARCHAR(50),
    POLICY_TYPE VARCHAR(50),
    EFFECTIVE_DATE DATE,
    EXPIRATION_DATE DATE,
    POLICY_STATUS VARCHAR(20), 
    ANNUAL_PREMIUM_USD NUMERIC(12, 2),
    PREMIUM_EARNED_USD NUMERIC(12, 2)
);

CREATE OR REPLACE TABLE FACT_CLAIMS (
    POLICY_KEY INT REFERENCES DIM_POLICY (POLICY_KEY),
    BUSINESS_KEY INT REFERENCES DIM_BUSINESS (BUSINESS_KEY),
    CLAIM_ID VARCHAR(50),
    CLAIM_DATE DATE,
    LOSS_TYPE VARCHAR(100),
    REPORTED_LAG_DAYS INT,
    INCURRED_AMOUNT_USD NUMERIC(12, 2),
    IS_CATASTROPHE BOOLEAN,
    CLAIM_STATUS VARCHAR(20)
);

### 1.2 - Insert data

In [ ]:
-- Populate DIM_BUSINESS (Local Cincinnati Focus)
INSERT INTO DIM_BUSINESS (BUSINESS_KEY, BUSINESS_NAME, BUSINESS_CITY, BUSINESS_STATE, NAICS_CODE, INDUSTRY_SEGMENT, ASSET_VALUE_MILLIONS)
VALUES
(1, 'Deeper Roots Coffee', 'Cincinnati', 'OH', '722515', 'Restaurant & Hospitality', 2.50),
(2, 'The Eagle OTR', 'Cincinnati', 'OH', '722511', 'Restaurant & Hospitality', 4.10),
(3, 'Rhinegeist Brewery', 'Cincinnati', 'OH', '312120', 'Beverage Manufacturing', 45.00),
(4, 'Cincinnati Zoo & Botanical Garden', 'Cincinnati', 'OH', '712130', 'Arts, Entertainment', 120.00),
(5, 'Local Food Truck Corp.', 'Cincinnati', 'OH', '722330', 'Mobile Food Services', 0.85),
(6, 'Queen City T-Shirt Printing', 'Cincinnati', 'OH', '323113', 'Small-Scale Manufacturing', 3.20),
(7, 'The Lytle Park Hotel', 'Cincinnati', 'OH', '721110', 'Hospitality', 65.00),
(8, 'Cincinnati Museum Center', 'Cincinnati', 'OH', '712110', 'Museums & Historical Sites', 95.50);

-- Set the Current Date for the Demo: October 2025
-- Policies 101 and 102 are expiring soon (next 30-60 days)

INSERT INTO DIM_POLICY (POLICY_KEY, BUSINESS_KEY, POLICY_ID, POLICY_TYPE, EFFECTIVE_DATE, EXPIRATION_DATE, POLICY_STATUS, ANNUAL_PREMIUM_USD, PREMIUM_EARNED_USD)
VALUES
(101, 1, 'GL00101', 'General Liability', '2024-11-01', '2025-11-01', 'Active', 12000.00, 12000.00), -- Deeper Roots (Expiring Soon)
(102, 2, 'GL00102', 'General Liability', '2024-10-25', '2025-10-25', 'Active', 18000.00, 18000.00), -- The Eagle OTR (Expiring Soon)
(103, 3, 'PC00103', 'Commercial Property', '2025-01-01', '2026-01-01', 'Active', 55000.00, 45833.33),  -- Premium earned for 10 months
(104, 4, 'WC00104', 'Workers Compensation', '2025-06-01', '2026-06-01', 'Active', 75000.00, 25000.00),  -- Premium earned for 4 months
(105, 5, 'GL00105', 'General Liability', '2024-11-01', '2025-11-01', 'Active', 4500.00, 4500.00), -- Food Truck (Expiring Soon, but Low Risk)
(106, 6, 'GL00106', 'General Liability', '2025-02-01', '2026-02-01', 'Active', 8500.00, 6375.00),  -- Premium earned for 9 months
(107, 7, 'PC00107', 'Commercial Property', '2025-04-01', '2026-04-01', 'Active', 95000.00, 55416.67),  -- Premium earned for 7 months
(108, 8, 'CY00108', 'Cyber Liability', '2025-01-01', '2026-01-01', 'Active', 15000.00, 12500.00);  -- Premium earned for 10 months

-- Populate FACT_CLAIMS (Claims to create high loss ratio for local companies)
INSERT INTO FACT_CLAIMS (POLICY_KEY, BUSINESS_KEY, CLAIM_ID, CLAIM_DATE, LOSS_TYPE, REPORTED_LAG_DAYS, INCURRED_AMOUNT_USD, IS_CATASTROPHE, CLAIM_STATUS)
VALUES
-- HIGH RISK: Deeper Roots Coffee (BUSINESS_KEY=1, POLICY_KEY=101) - LR = 1.50
(101, 1, 'C10001', '2025-03-10', 'Slip & Fall', 1, 8000.00, FALSE, 'Closed'),
(101, 1, 'C10002', '2025-05-05', 'Property Damage (burst pipe)', 3, 5500.00, FALSE, 'Closed'),
(101, 1, 'C10003', '2025-08-25', 'Customer Injury (Hot beverage)', 1, 4500.00, FALSE, 'Open'),
-- HIGH RISK: The Eagle OTR (BUSINESS_KEY=2, POLICY_KEY=102) - LR = 1.50
(102, 2, 'C20001', '2025-01-05', 'Food Poisoning (Single incident)', 2, 10000.00, FALSE, 'Closed'),
(102, 2, 'C20002', '2025-04-10', 'Liquor Liability', 10, 15000.00, FALSE, 'Open'),
(102, 2, 'C20003', '2025-07-20', 'Theft (patio furniture)', 4, 2000.00, FALSE, 'Closed'),

-- Other claims for context
(103, 3, 'C30001', '2025-03-20', 'Minor Fire Damage', 7, 12000.00, FALSE, 'Closed'), -- Rhinegeist Brewery
(104, 4, 'C40001', '2025-07-01', 'Workplace Minor Injury', 2, 5500.00, FALSE, 'Closed'), -- Cincinnati Zoo
(105, 5, 'C50001', '2025-02-05', 'Vehicle Collision', 5, 2500.00, FALSE, 'Open'), -- Local Food Truck Corp.
(106, 6, 'C60001', '2025-05-01', 'Equipment Failure', 3, 8000.00, FALSE, 'Open'), -- Queen City T-Shirt Printing
(107, 7, 'C70001', '2025-08-03', 'Small Hail Damage', 2, 15000.00, FALSE, 'Open'), -- The Lytle Park Hotel
(108, 8, 'C80001', '2025-09-10', 'System Ransomware', 1, 10000.00, FALSE, 'Open'); -- Cincinnati Museum Center

### 1.3 - Run some sample queries

In [ ]:
-- Amount of loss incurred from claims for Commerical Property policies
SELECT policy_type, sum(incurred_amount_usd)
FROM fact_claims c
JOIN dim_policy p
  on c.policy_key = p.policy_key
WHERE policy_type = 'Commercial Property'
GROUP BY policy_type
;

-- Run some other queries!

## Section 2: Cortex Analyst

Cortex Analyst is a fully managed, AI-powered feature in Snowflake to answer natural language queries about your structured data. It relies on a Semantic model to provide metadata and context about your relational data to the LLM so that it can generate SQL based on the question at hand.

The Semantic Layer can be defined as a YAML file or as a Semantic View - a new schema-level object in Snowflake.


### 2.1 - Semantic Views
Semantic Views are schema-level objects in Snowflake that enable us to define business metrics, entities, and their relationships. This context will be leveraged by Cortex Analyst for generating SQL to answer natural language questions.

Semantic Views are comprised of the following components:

- **Tables**: logical tables that map to Snowflake tables or views
- **Facts**: row-level attributes tied to a logical table that represent specific business events or transactions
- **Dimensions**: categorical attributes tied to a logical table that gives meaning to metrics by grouping data into meaningful categories
- **Metrics**: quantifiable measures of business performance calculated by aggregating facts or other columns from the same table
**Named filters**: logic to filter a logical table based on some business rule
**Relationships**: how logical tables are mapped to one another. This enables Cortex Analyst to join multiple logical tables together

Semantic Views can be created through SQL (as seen below), but also through the Snowsight UI which is what we'll walk through together in this lab.

In [ ]:
-- -- Semantic View SQL - We will create this together via the UI
-- create or replace semantic view CORTEX_LAB_DB.RISK_DATA.RISK_SEMANTIC_VIEW tables (
--     DIM_BUSINESS primary key (BUSINESS_KEY) comment = 'This table stores information about businesses, including a unique identifier, name, location, industry classification (e.g., Restaurant & Hospitality), and financial data, to support business intelligence and analytics.',
--     DIM_POLICY primary key (POLICY_KEY) comment = 'This table stores information about insurance policies. CRITICAL FIELDS: POLICY_STATUS ("Active"), EXPIRATION_DATE (to check for upcoming renewals), and PREMIUM_EARNED_USD (for Loss Ratio calculation).',
--     FACT_CLAIMS primary key (CLAIM_ID) comment = 'This table stores information about insurance claims, including the policy and business keys, incurred amount, and claim status. CLAIM_ID is the unique identifier for a claim.'
-- ) relationships (
--     CLAIMS_TO_POLICIES as FACT_CLAIMS(POLICY_KEY) references DIM_POLICY(POLICY_KEY),
--     CLAIMS_TO_BUSINESSES as FACT_CLAIMS(BUSINESS_KEY) references DIM_BUSINESS(BUSINESS_KEY)
-- ) facts (
--     DIM_BUSINESS.ASSET_VALUE_MILLIONS as ASSET_VALUE_MILLIONS comment = 'The total value of assets held by the business, expressed in millions of dollars.',
--     DIM_BUSINESS.BUSINESS_KEY as BUSINESS_KEY comment = 'Unique identifier for a business entity.',
--     DIM_POLICY.ANNUAL_PREMIUM_USD as ANNUAL_PREMIUM_USD comment = 'The total annual premium amount paid by policyholders in US dollars.',
--     DIM_POLICY.PREMIUM_EARNED_USD as PREMIUM_EARNED_USD comment = 'The portion of the premium that has been recognized as revenue, critical for calculating accurate Loss Ratio (Incurred Claims / Earned Premium).',
--     DIM_POLICY.POLICY_KEY as POLICY_KEY comment = 'Unique identifier for a policy in the insurance portfolio.',
--     FACT_CLAIMS.BUSINESS_KEY as BUSINESS_KEY comment = 'Unique identifier for a business entity, links claims to business.',
--     FACT_CLAIMS.CLAIM_ID as CLAIM_ID comment = 'Unique identifier for a claim submitted by a customer.',
--     FACT_CLAIMS.INCURRED_AMOUNT_USD as INCURRED_AMOUNT_USD comment = 'The total amount of financial loss incurred for a claim, expressed in US dollars. Used as the numerator in the Loss Ratio calculation.',
--     FACT_CLAIMS.POLICY_KEY as POLICY_KEY comment = 'Unique identifier for a policy, links claims to policy details.',
--     FACT_CLAIMS.REPORTED_LAG_DAYS as REPORTED_LAG_DAYS comment = 'The number of days between the date an incident occurred and the date it was reported.'
-- ) dimensions (
--     DIM_BUSINESS.BUSINESS_CITY as BUSINESS_CITY comment = 'The city where the business is located. Example: Cincinnati.',
--     DIM_BUSINESS.BUSINESS_NAME as BUSINESS_NAME comment = 'The name of the business or organization.',
--     DIM_BUSINESS.BUSINESS_STATE as BUSINESS_STATE comment = 'The state in which the business is located.',
--     DIM_BUSINESS.INDUSTRY_SEGMENT as INDUSTRY_SEGMENT comment = 'The industry segment in which the business operates, such as Restaurant & Hospitality, Beverage Manufacturing, or Arts, Entertainment.',
--     DIM_BUSINESS.NAICS_CODE as NAICS_CODE comment = 'Industry Classification Code.',
--     DIM_POLICY.EFFECTIVE_DATE as EFFECTIVE_DATE comment = 'The date when a policy becomes effective.',
--     DIM_POLICY.EXPIRATION_DATE as EXPIRATION_DATE comment = 'Date on which the policy expires. Use this to find policies due for renewal soon (e.g., within 60 days of today).',
--     DIM_POLICY.POLICY_ID as POLICY_ID comment = 'Unique identifier for a policy.',
--     DIM_POLICY.POLICY_STATUS as POLICY_STATUS comment = 'The current status of the policy. Must be "Active" to be included in current risk reviews.',
--     DIM_POLICY.POLICY_TYPE as POLICY_TYPE comment = 'Type of insurance policy held by the customer, such as General Liability, Commercial Property, or Workers Compensation.',
--     FACT_CLAIMS.CLAIM_DATE as CLAIM_DATE comment = 'Date on which the claim was made.',
--     FACT_CLAIMS.CLAIM_STATUS as CLAIM_STATUS comment = 'The current status of a claim, indicating whether it is Open or Closed.',
--     FACT_CLAIMS.IS_CATASTROPHE as IS_CATASTROPHE comment = 'Indicates whether the claim is related to a catastrophic event.',
--     FACT_CLAIMS.LOSS_TYPE as LOSS_TYPE comment = 'The type of loss or damage that occurred.'
-- ) with extension (
--     CA = '{"tables":[{"name":"DIM_BUSINESS","dimensions":[{"name":"BUSINESS_CITY","sample_values":["Cincinnati"]},{"name":"BUSINESS_NAME","sample_values":["Deeper Roots Coffee","The Eagle OTR","Rhinegeist Brewery"]},{"name":"BUSINESS_STATE","sample_values":["OH"]},{"name":"INDUSTRY_SEGMENT","sample_values":["Restaurant & Hospitality","Beverage Manufacturing","Arts, Entertainment"]},{"name":"NAICS_CODE","sample_values":["722515","722511","312120"]}],"facts":[{"name":"ASSET_VALUE_MILLIONS","sample_values":["2.50","4.10","45.00"]},{"name":"BUSINESS_KEY","sample_values":["1","2","3"]}]},{"name":"DIM_POLICY","dimensions":[{"name":"POLICY_ID","sample_values":["GL00101","GL00102","PC00103"]},{"name":"POLICY_TYPE","sample_values":["General Liability","Commercial Property"]},{"name":"POLICY_STATUS","sample_values":["Active","Inactive"]}],"facts":[{"name":"ANNUAL_PREMIUM_USD","sample_values":["12000.00","18000.00","55000.00"]},{"name":"PREMIUM_EARNED_USD","sample_values":["12000.00","18000.00","45833.33"]},{"name":"POLICY_KEY","sample_values":["101","102","103"]}],"time_dimensions":[{"name":"EFFECTIVE_DATE","sample_values":["2024-11-01","2024-10-25","2025-01-01"]},{"name":"EXPIRATION_DATE","sample_values":["2025-11-01","2025-10-25","2026-01-01"]}]},{"name":"FACT_CLAIMS","dimensions":[{"name":"CLAIM_STATUS","sample_values":["Open","Closed"]},{"name":"IS_CATASTROPHE","sample_values":["FALSE"]},{"name":"LOSS_TYPE","sample_values":["Slip & Fall","Liquor Liability","Food Poisoning"]}],"facts":[{"name":"BUSINESS_KEY","sample_values":["1","2","3"]},{"name":"CLAIM_ID","sample_values":["C10003","C20002","C10001"]},{"name":"INCURRED_AMOUNT_USD","sample_values":["4500.00","15000.00","8000.00"]},{"name":"POLICY_KEY","sample_values":["101","102","103"]},{"name":"REPORTED_LAG_DAYS","sample_values":["1","10","3"]}],"time_dimensions":[{"name":"CLAIM_DATE","sample_values":["2025-08-25","2025-04-10","2025-03-10"]}]}],"relationships":[{"name":"CLAIMS_TO_POLICIES"},{"name":"CLAIMS_TO_BUSINESSES"}]}'
-- );


In [ ]:
-- Verify Semantic View
SHOW SEMANTIC VIEWS;

### 2.2 - Creating our first Cortex Agent
Now that we have a Semantic View created for use with Cortex Analyst, we can create a Cortex Agent that will be able to answer natural language questions about our data.

Cortex Agents are the brains of this demo. Agents will orchestrate both structured and unstructured data to deliver insights. They plan tasks, use tools to execute these tasks, and generate responses. Agents can be configured to use Cortex Analyst (structured data), Cortex Search (unstructured data), and custom tools to generate answers.



In [ ]:
CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Company_Chatbot_Agent_Structured
WITH PROFILE='{"display_name": "Risk Analysis Agent"}'
COMMENT=$$ This is an agent that can answer questions about our insurance data including policies, businesses, and claims. $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "You are a data analyst who has access to insurance policies, insured businesses, and claims. If user does not specify a date range assume it for year 2025. Leverage data from all domains to analyse & answer user questions. Provide visualizations if possible. Trendlines should default to linecharts, Categories default to Barchart.",
    "orchestration": "Use cortex analyst for any questions regarding the policies, businesses, and claims data.\n\n\n",
    "sample_questions": [
      {
        "question": "What has been our incurred loss by category over the last 12 months?"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Insurance Datamart",
        "description": "Allows users to query finance data for a company in terms of revenue & expenses."
      }
    }
  ],
  "tool_resources": {
    "Query Insurance Datamart": {
      "semantic_view": "CORTEX_LAB_DB.RISK_DATA.RISK_SEMANTIC_VIEW"
    }
  }
}
$$;

### 2.3 Test Analyst Agent v1!

Open Snowflake Intelligence by clicking on 'AI & ML' on the left navbar and then choosing 'Snowflake Intelligence'.

Once there, ask away about our structured data! Here are some sample questions as thought starters:

- What was our incurred loss amount by policy type over the last 12 months?
- How many claims have been made over the last 3 months? 


## Section 3: Custom Tools

In addition to Cortex Analyst and Cortex Search, we can also equip our agent with custom tools for execution. In this section, we'll create additional tools for sending emails and performing web search.

### 3.1 Email Tool
Below, we define a stored procedure for sending emails from Snowflake Intelligence. This will enable us to create emails on interesting insights that are derived from Snowflake Intelligence, all without ever leaving the platform.

In [ ]:
CREATE OR REPLACE PROCEDURE send_mail(recipient STRING, subject STRING, text STRING)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.11'
PACKAGES = ('snowflake-snowpark-python')
HANDLER = 'send_mail'
AS
$$
def send_mail(session, recipient, subject, text):
    # Use SQL execution, not session.call
    session.sql(f"""
        CALL SYSTEM$SEND_EMAIL(
            'ai_email_int',
            '{recipient}',
            '{subject}',
            '{text}',
            'text/html'
        )
    """).collect()

    return f'Email was sent to {recipient} with subject: "{subject}".'
$$;


### 3.2 Web Search Tool
We can also create functions using python and make those available as tools to the agent. The code below creates a simple web search function that can be used by our agent for getting additional information from the web.

In [ ]:
CREATE OR REPLACE FUNCTION Web_search(search_query STRING)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.11
ARTIFACT_REPOSITORY = snowflake.snowpark.pypi_shared_repository
HANDLER = 'web_search'
EXTERNAL_ACCESS_INTEGRATIONS = (Snowflake_intelligence_ExternalAccess_Integration)
PACKAGES = ('tavily-python')
AS
$$
from tavily import TavilyClient

def web_search(search_query: str) -> str:
    TAVILY_API_KEY = ""
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    response = tavily_client.search(query=search_query)

    # Return the first snippet (if available)
    for result in response.get('results', []):
        return result.get('content', '')[:1000]

    return "No results found."
$$;


### 3.3 Update the Agent with tools
Now that we have the tools defined, we can recreate our agent to give it the ability to call the tools.

In [ ]:
-- Tavily version
CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Company_Chatbot_Agent_Tools
WITH PROFILE='{"display_name": "2-Risk Chatbot - Tools"}'
COMMENT=$$ This is an agent that can answer questions about our insurance data including policies, businesses, and claims. $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "You are a data analyst who has access to insurance policies, insured businesses, and claims. If user does not specify a date range assume it for year 2025. Leverage data from all domains to analyze & answer user questions. Provide visualizations if possible. Trendlines should default to linecharts, Categories default to Barchart. You have access to a web_search tool which can search all of the web to find information, such as yelp reviews.",
    "orchestration": "Use cortex analyst for any questions regarding the policies, businesses, and claims data.\n\n\n",
    "sample_questions": [
      {
        "question": "What was our incurred loss amount by policy type over the last 12 months?"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Insurance Datamart",
        "description": "Allows users to query finance data for a company in terms of revenue & expenses."
      }
    },
    {
      "tool_spec": {
        "type": "generic",
        "name": "Web_search",
        "description": "This tool should be used if the user wants to search the web for additional information about the business they're looking to evaluate.",
        "input_schema": {
          "type": "object",
          "properties": {
            "search_query": {
              "description": "The user provided search query.",
              "type": "string"
            }
          },
          "required": [
            "search_query"
          ]
        }
      }
    },
    {
      "tool_spec": {
        "type": "generic",
        "name": "Send_Emails",
        "description": "This tool is used to send emails to a email recipient. It can take an email, subject & content as input to send the email. Always use HTML formatted content for the emails.",
        "input_schema": {
          "type": "object",
          "properties": {
            "recipient": {
              "description": "recipient of email",
              "type": "string"
            },
            "subject": {
              "description": "subject of email",
              "type": "string"
            },
            "text": {
              "description": "content of email",
              "type": "string"
            }
          },
          "required": [
            "text",
            "recipient",
            "subject"
          ]
        }
      }
    }
  ],
  "tool_resources": {
    "Query Insurance Datamart": {
      "semantic_view": "CORTEX_LAB_DB.RISK_DATA.RISK_SEMANTIC_VIEW"
    },
    "Send_Emails": {
      "execution_environment": {
        "query_timeout": 0,
        "type": "warehouse",
        "warehouse": "SNOW_INTELLIGENCE_DEMO_WH"
      },
      "identifier": "CORTEX_LAB_DB.RISK_DATA.SEND_MAIL",
      "name": "SEND_MAIL(VARCHAR, VARCHAR, VARCHAR)",
      "type": "procedure"
    },
    "Web_search": {
      "execution_environment": {
        "query_timeout": 0,
        "type": "warehouse",
        "warehouse": "SNOW_INTELLIGENCE_DEMO_WH"
      },
      "identifier": "CORTEX_LAB_DB.RISK_DATA.WEB_SEARCH",
      "name": "WEB_SEARCH(VARCHAR)",
      "type": "function"
    }
  }
}
$$;

### 3.4 Test the updated agent

One sample flow (ask all questions in one conversation):
- Show me all active General Liability policies for our clients in Cincinnati's Restaurant & Hospitality sector with a Loss Ratio above 1.4 that are due for renewal soon. Calculate the Loss Ratio.
- Start with The Eagle OTR. Use the web search tool to find any recent news or regulatory actions related to health code violations or labor issues in Cincinnati.
- Combine the internal policy data (loss ratio) with the external web search findings and provide a final renewal recommendation for The Eagle OTR. Justify your score.

## Section 4: AI Observability

After asking a few questions to our agent, we can explore Snowflake's built in AI Observability tooling to understand what tools were invoked to accomplish the given tasks.

Start by clicking on 'AI & ML' on the left hand havbar, and then opening 'Cortex Agents' in a new tab. Once on the Cortex Agents screen, choose the 'COMPANY_CHATBOT_AGENT_TOOLS' agent. Here, you can see all of the configuration for our agent, including which tools it has available.

Next, click on 'Monitoring' and then choose one of your conversations. This is where we can see all the detail going on behind the scenes. Find a task that used Cortex Analyst - there we can see the exact SQL that was generated and executed to answer the given question. On the 'Web search' tasks, we can see the given prompt and the web search results that was fed back to the model.

Additional potential features:
- Is document upload via Intelligence possible? Would be cool to upload a doc, parse it, fetch relevant info from web + and related data in our tables.
- Cortex search for docs
- Calling an ML model via a tool

## Evals